In [1]:
import gymnasium as gym

env = gym.make("room_env:RoomEnv-v0")
(observation, question), info = env.reset()
rewards = 0

while True:
    (observation, question), reward, done, truncated, info = env.step(
        "This is my answer!"
    )
    rewards += reward
    if done:
        break

print(rewards)


/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/envs/registration.py:578: UserWarning: WARN: The environment RoomEnv-v0 is out of date. You should consider upgrading to version `v2`.
  logger.warn(
/home/tk/.virtualenvs/human-memory/lib/python3.9/site-packages/gymnasium/envs/registration.py:435: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes']
  logger.warn(
2023-08-07 16:00:23.999 WARNING room0 - __init__: The obserations will be limited by {'heads': None, 'tails': None, 'names': None, 'allow_spaces': False}
2023-08-07 16:00:24.000 WARNING room0 - load_semantic_knowledge: Tails that are heads will be removed.
2023-08-07 16:00:24.001 INF

0


In [4]:
import gymnasium as gym

env = gym.make("room_env:RoomEnv-v1")
(observation, question), info = env.reset()
rewards = 0

while True:
    # There is one different thing in the RoomEnv from the original AAAI-2023 paper:
    # The reward is either +1 or -1, instead of +1 or 0.
    (observation, question), reward, done, truncated, info = env.step("my answer!")
    rewards += reward
    if done:
        break

# print(rewards)

In [2]:
observation, question


({'human': 'Alexander',
  'object': 'dog',
  'object_location': 'kennel',
  'current_time': 128},
 None)

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True

from pprint import pprint
from room_env.utils import get_handcrafted


results = get_handcrafted(
    env="RoomEnv-v1",
    des_size="l",
    seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    question_prob=1.0,
    policies={
        "memory_management": "rl",
        "question_answer": "episodic_semantic",
        "encoding": "argmax",
    },
    capacities=[2, 4, 8, 16, 32, 64],
    allow_random_human=False,
    allow_random_question=False,
    varying_rewards=False,
    check_resources=True,
)
pprint(results)

In [ ]:
import logging

logger = logging.getLogger()
logger.disabled = True


import gymnasium as gym
import room_env

for capacity in [2, 4, 8, 16, 32, 64]:
    for question_prob in [0.5, 1.0]:
        for pretrain_semantic in [False, True]:
            env = gym.make(
                "RoomEnv-v1",
                capacity={
                    "episodic": capacity // 2,
                    "semantic": capacity // 2,
                    "short": 1,
                },
                question_prob=question_prob,
                pretrain_semantic=pretrain_semantic,
            )
            observation, info = env.reset()
            while True:
                observation, reward, done, truncated, info = env.step(0)
                if done:
                    break